# Flight Lookup Assistant


In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

## 01. Define local function


In [2]:
import requests
import os

FLIGHT_API_KEY = "<key from aviationstack.com>" # os.getenv("FLIGHT_API_KEY")

def get_flight_details(flight_number):
    url = f"http://api.aviationstack.com/v1/flights?access_key={FLIGHT_API_KEY}&flight_iata={flight_number}"
    response = requests.get(url=url)
    
    flight_data = response.json()["data"][0]
    flight_status = flight_data["flight_status"]
    
    actual_landing_date = flight_data["arrival"]["actual"]
    estimated_landing_date = flight_data["arrival"]["estimated"]
    airline = flight_data["airline"]["name"]
    
    return f"""Flight: {flight_number}
        - Airline: {airline}
        - Flight status: {flight_status}
        - Estimated arrival: {estimated_landing_date}
        - Actual arrival: {actual_landing_date}
    """

## 02. Create an Assistant


In [5]:
assistant = client.beta.assistants.create(
    name="Flight Bot",    
    instructions="You are a flight information bot. Use the provided functions to answer questions.",
    model="gpt-4-1106-preview",
    tools=[{
        "type": "function",
        "function": {
            "name": "get_flight_details",
            "description": "This function looks up flight details information",
            "parameters": {
                "type": "object",
                "properties": {
                "flight_number": {
                    "type": "string",
                    "description": "The flight number"
                }
                },
                "required": ["flight_number"]
            }
        }
    }]
)
assistant.id

'asst_YoUYLAS6hvgWYbBol4p2ZSWt'

## 03. Create Conversational Thread


In [6]:
# Create a thread

thread = client.beta.threads.create()
thread

Thread(id='thread_jXWCSEgAUQW70ktbgGbxrDun', created_at=1702871275, metadata={}, object='thread')

## 04. Instantiate a Run Object


In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, 
    instructions="When is flight UA2042 arriving?"
)
run

Run(id='run_cwAXySYI4S854LxYzgH1M44w', assistant_id='asst_YoUYLAS6hvgWYbBol4p2ZSWt', cancelled_at=None, completed_at=None, created_at=1702871328, expires_at=1702871928, failed_at=None, file_ids=[], instructions='When is flight UA2042 arriving?', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_jXWCSEgAUQW70ktbgGbxrDun', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_flight_details', parameters={'type': 'object', 'properties': {'flight_number': {'type': 'string', 'description': 'The flight number'}}, 'required': ['flight_number']}, description='This function looks up flight details information'), type='function')])

In [8]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run.status

'requires_action'

## 05. Divert the run to call local function


In [9]:
import json

def get_outputs_for_tool_call(tool_call):
    flight_number = json.loads(tool_call.function.arguments)["flight_number"]
    flight_details = get_flight_details(flight_number)
    return {
        "tool_call_id": tool_call.id,
        "output": flight_details
    }

In [10]:
run

Run(id='run_cwAXySYI4S854LxYzgH1M44w', assistant_id='asst_YoUYLAS6hvgWYbBol4p2ZSWt', cancelled_at=None, completed_at=None, created_at=1702871328, expires_at=1702871928, failed_at=None, file_ids=[], instructions='When is flight UA2042 arriving?', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_gMIwXLRJ8iLsyEbNFgTWHHKF', function=Function(arguments='{"flight_number": "UA2042"}', name='get_flight_details'), type='function')]), type='submit_tool_outputs'), started_at=1702871328, status='requires_action', thread_id='thread_jXWCSEgAUQW70ktbgGbxrDun', tools=[ToolAssistantToolsFunction(function=FunctionDefinition(name='get_flight_details', parameters={'type': 'object', 'properties': {'flight_number': {'type': 'string', 'description': 'The flight number'}}, 'required': ['flight_number']}, description='This function looks up flight 

In [11]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls

In [12]:
tool_calls

[RequiredActionFunctionToolCall(id='call_gMIwXLRJ8iLsyEbNFgTWHHKF', function=Function(arguments='{"flight_number": "UA2042"}', name='get_flight_details'), type='function')]

In [13]:
tool_outputs = map(get_outputs_for_tool_call, tool_calls)

In [14]:
list(tool_outputs)

[{'tool_call_id': 'call_gMIwXLRJ8iLsyEbNFgTWHHKF',
  'output': 'Flight: UA2042\n        - Airline: United Airlines\n        - Flight status: landed\n        - Estimated arrival: 2023-12-17T13:00:00+00:00\n        - Actual arrival: 2023-12-17T13:07:00+00:00\n    '}]